In [ ]:
#libraries
import pandas as pd
import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs
#import pprint    # library for cleanly printing Python data structures
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import numpy as np
import geopandas as gpd
import utm
import contextily as cx
import folium
import plotly
#import mpld3
from PIL import Image, ImageFont, ImageDraw
from pyproj import Proj
from shapely.geometry import Point
from geopandas import GeoDataFrame
from folium import plugins
from matplotlib.pyplot import figure
 

# %matplotlib inline
# from matplotlib import rcParams
# pd.options.mode.chained_assignment = None

# pp = pprint.PrettyPrinter()


In [ ]:
########IMPORTING INCOME SETOR CENSITARIO
import basedosdados as bd
              
# Uploading setor censitario data abour income
df_income = bd.read_table(dataset_id='br_ibge_censo_demografico', 
table_id='setor_censitario_domicilio_renda_2010',
billing_project_id="cyplan255", reauth=True)


#dicionário de dados: https://basedosdados.org/dataset/br-ibge-censo-demografico

#Reduzir base para o município de SP
SET_CEN_SP_income = df_income[df_income["sigla_uf"]== "SP1"]


#Total do rendimento nominal mensal dos domicílios particulares permanents - v003
#renaming v002 as income
SET_CEN_SP_income.rename(columns = {'v003':'total_income'}, inplace = True)



########IMPORTING HOUSEHOLDS SETOR CENSITARIO
import basedosdados as bd
              
# Uploading setor censitario data about domicilios
df_domicilios = bd.read_table(dataset_id='br_ibge_censo_demografico', 
table_id='setor_censitario_domicilio_caracteristicas_gerais_2010',
billing_project_id="cyplan255", reauth=True)


#Reduzir base para o município de SP
SET_CEN_SP_domicilios = df_domicilios[df_domicilios["sigla_uf"]== "SP1"]


#Domicílios particulares permanentes - v002
#renaming v002 as households
SET_CEN_SP_domicilios.rename(columns = {'v002':'total_domicilios'}, inplace = True)


########## Importing shapefiles setor censitário#####

#import shapefile using geopandas
setor_censitario_SPS = gpd.read_file('data/sp_setores_censitarios/35SEE250GC_SIR.shp')


#reducing to only sp municipality
setor_censitario_sp = setor_censitario_SPS[setor_censitario_SPS["NM_MUNICIP"]== "SÃO PAULO"]


#renaming column setor censitario
setor_censitario_sp.rename(columns = {'CD_GEOCODI':'id_setor_censitario'}, inplace = True)


#Merging with the income dataframe
setor_censitario_sp_income = setor_censitario_sp.merge(SET_CEN_SP_income, on='id_setor_censitario')



#Merging with the domiciolios dataframe
setor_censitario_sp_inc_dom = setor_censitario_sp_income.merge(SET_CEN_SP_domicilios, on='id_setor_censitario')


#column names
list(setor_censitario_sp_inc_dom.columns)

#Creating a new column: household income (rendimento nominal medio mensal)
setor_censitario_sp_inc_dom["income_household"]= (setor_censitario_sp_inc_dom["total_income"]/setor_censitario_sp_inc_dom["total_domicilios"])

#investgating the columns
setor_censitario_sp_inc_dom["total_domicilios"].describe()

#replacing infinte values
setor_censitario_sp_inc_dom.replace([np.inf, -np.inf], np.nan, inplace=True)

#investgating the columns
setor_censitario_sp_inc_dom["income_household"].describe()

#list(setor_censitario_sp_inc_dom.columns)
setor_censitario_sp_inc_dom = setor_censitario_sp_inc_dom[['id_setor_censitario','NM_DISTRIT','geometry','income_household']]


#ordem decrescente
setor_censitario_sp_inc_dom.sort_values(by=['income_household'], ascending=False)



In [ ]:
#fundurb_housing_geo
#uploading only valid geometries
fundurb_housing_geo_2 = pd.read_csv('fundurb_housing_geo_2.csv')  
fundurb_housing_geo_2


crs = {'init': 'epsg:4326'}

geometry = [Point(xy)for xy in zip (fundurb_housing_geo_2["longitude"], fundurb_housing_geo_2["latitude"])]
geometry[:3]

fundurb_housing_geo_2 = gpd.GeoDataFrame(fundurb_housing_geo_2, crs=crs, geometry=geometry)


In [ ]:
#PLOTTING THE MAP
sp_map = folium.Map(location=[-23.533773, -46.625290], zoom_start=11, tiles='Cartodb positron')

myscale = (setor_censitario_sp_inc_dom['income_household'].quantile((0,0.25,0.5,0.75,0.95,1))).tolist()

sp_map.choropleth(
 geo_data=setor_censitario_sp_inc_dom,
 name='Choropleth',
 data=setor_censitario_sp_inc_dom,
 columns=["id_setor_censitario", "income_household"],
 key_on="feature.properties.id_setor_censitario",
 fill_color='YlGnBu',
 fill_opacity=0.6,
 line_opacity=0.2,
 legend_name='Average income per household',
 smooth_factor=0,
 bins=[85,2500,5000,10000, 30000, 103319],
 nan_fill_color = "grey"
)

oodc_hab = [[point.xy[1][0], point.xy[0][0]] for point in fundurb_housing_geo_2.geometry]


plugins.MarkerCluster(oodc_hab).add_to(sp_map)

sp_map